# Costarican Household poverty level prediction

Prediksi level kemiskinan di kostarika

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 138) # untuk set berapa jumlah kolom yang ditampilkan saat memanggil perintah .dtypes dan .head
data=pd.read_csv('D:/Project/Data Science Project/Datasets/Costarica-extreme-poverty/train.csv')

In [2]:
data.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,NaN,0,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,ID_68de51c94,NaN,0,8,0,1,1,0,NaN,0,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,81,289,16,121,4,1.777778,1.0,121.0,289,4
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,4


In [3]:
data['v2a1'].value_counts(bins=3)
c=data['v2a1'].isnull().sum()
print(c)

6860


### Menghilangkan kolom id dan idhogar pada dataset 

In [4]:
data.drop(labels='Id', axis=1,inplace=True)
data.drop(labels='idhogar',axis=1, inplace=True)     

### Menghitung jumlah data yang hilang dari setiap kolom dataset

In [5]:
missing_data=data.isnull()
missing_data.head()
for column in missing_data.columns.values.tolist():
    column1=data[column]
    if column1.isnull().sum() >0: 
        print(column)
        print (missing_data[column].value_counts())
        print("")    

v2a1
True     6860
False    2697
Name: v2a1, dtype: int64

v18q1
True     7342
False    2215
Name: v18q1, dtype: int64

rez_esc
True     7928
False    1629
Name: rez_esc, dtype: int64

meaneduc
False    9552
True        5
Name: meaneduc, dtype: int64

SQBmeaned
False    9552
True        5
Name: SQBmeaned, dtype: int64



### Karena pada kolom v2a1,v18q1,rez_esc jumlah data yang hilang terlalu banyak maka kolom data tersebut  dihilangkan

In [6]:
column_to_drop=['v2a1','v18q1','rez_esc']
data.drop(labels=column_to_drop,axis=1,inplace=True)

### Pada kolom meaneduc dan SQBmeaned data yang hilang sedikit maka data yang hilang akan digantikan dari mean pada kolom tersebut

In [7]:
column_to_mean=['SQBmeaned','meaneduc']
avg1,avg2=data[column_to_mean].astype('float').mean(axis=0)
print(avg1, avg2)

102.58886699019641 9.231522935701435


In [8]:
data['SQBmeaned'].replace(np.nan,avg1,inplace=True)
data['meaneduc'].replace(np.nan,avg2,inplace=True)

In [9]:
data[column_to_mean].isnull().sum()

SQBmeaned    0
meaneduc     0
dtype: int64

### Pengolahan data pada kolom yang terdapat string dan numeric.

kolom-kolom tersebut adalah dependency, edjefe dan edjefa

In [10]:
mixed_data_column=['dependency','edjefe','edjefa']
data[mixed_data_column].dtypes

dependency    object
edjefe        object
edjefa        object
dtype: object

In [11]:
def string_to_number(x):
    if x=='yes':
        x=1
    elif x=='no':
        x=0
    return x
    
    

In [12]:
data['dependency']=data['dependency'].map(string_to_number) # mmengubah data string menjadi numerik (yes=1, no=0)
data['edjefe']=data['edjefe'].map(string_to_number)
data['edjefa']=data['edjefa'].map(string_to_number)
data['dependency']=data['dependency'].astype('float64',inplace=True) # mengubah data menjadi float
data['edjefe']=data['edjefe'].astype('float64',inplace=True) 
data['edjefa']=data['edjefa'].astype('float64',inplace=True) 

In [13]:
data['dependency']=data['dependency'].astype('float64',inplace=True)

In [14]:
data['dependency'].dtypes


dtype('float64')

In [15]:
data['edjefa'].value_counts(bins=3)

(-0.022000000000000002, 7.0]    7973
(7.0, 14.0]                     1193
(14.0, 21.0]                     391
Name: edjefa, dtype: int64

In [16]:
data.shape

(9557, 138)

### Lihat korelasi variabel dengan target


Dapat kita lihat melaui tabel dibawah bahwa kolom elimbasu5 mempunyai korelasi NaN. Sehingga kolom tersebut akan didrop

In [17]:
a=data.corr()['Target'][:]
a


hacdor            -0.191714
rooms              0.226208
hacapo            -0.138008
v14a               0.063382
refrig             0.126792
v18q               0.238864
r4h1              -0.229889
r4h2               0.101253
r4h3              -0.043359
r4m1              -0.253163
r4m2              -0.036959
r4m3              -0.173931
r4t1              -0.316745
r4t2               0.048904
r4t3              -0.147548
tamhog            -0.147642
tamviv            -0.158224
escolari           0.302305
hhsize            -0.147642
paredblolad        0.261274
paredzocalo       -0.079199
paredpreb         -0.099002
pareddes          -0.082923
paredmad          -0.168237
paredzinc         -0.051806
paredfibras       -0.027730
paredother        -0.000629
pisomoscer         0.280284
pisocemento       -0.205439
pisoother          0.021219
pisonatur         -0.044957
pisonotiene       -0.111270
pisomadera        -0.116306
techozinc          0.027652
techoentrepiso     0.019592
techocane         -0

In [18]:
data.drop(labels='elimbasu5',axis=1,inplace=True)

In [19]:
data.shape

(9557, 137)

### Drop kolom squares pada dataset

Karena data pada kolom squares merupakan kuadrat dari feature yang sudah ada maka kolom data tersebut kita hilangkan

In [20]:
data.dtypes

hacdor               int64
rooms                int64
hacapo               int64
v14a                 int64
refrig               int64
v18q                 int64
r4h1                 int64
r4h2                 int64
r4h3                 int64
r4m1                 int64
r4m2                 int64
r4m3                 int64
r4t1                 int64
r4t2                 int64
r4t3                 int64
tamhog               int64
tamviv               int64
escolari             int64
hhsize               int64
paredblolad          int64
paredzocalo          int64
paredpreb            int64
pareddes             int64
paredmad             int64
paredzinc            int64
paredfibras          int64
paredother           int64
pisomoscer           int64
pisocemento          int64
pisoother            int64
pisonatur            int64
pisonotiene          int64
pisomadera           int64
techozinc            int64
techoentrepiso       int64
techocane            int64
techootro            int64
c

In [21]:
useless_data=['SQBescolari','SQBage','SQBhogar_total','SQBedjefe','SQBhogar_nin','SQBovercrowding','SQBdependency','SQBmeaned','agesq']
for a in useless_data:
    data.drop(labels=a,axis=1,inplace=True)

In [22]:
data.shape

(9557, 128)

### Standarisasi Data 

In [23]:
y=data['Target']
X=data.drop(labels='Target',axis=1)

In [24]:
list_kolom=X.columns.values.tolist()
for i in list_kolom:
    print(i)

hacdor
rooms
hacapo
v14a
refrig
v18q
r4h1
r4h2
r4h3
r4m1
r4m2
r4m3
r4t1
r4t2
r4t3
tamhog
tamviv
escolari
hhsize
paredblolad
paredzocalo
paredpreb
pareddes
paredmad
paredzinc
paredfibras
paredother
pisomoscer
pisocemento
pisoother
pisonatur
pisonotiene
pisomadera
techozinc
techoentrepiso
techocane
techootro
cielorazo
abastaguadentro
abastaguafuera
abastaguano
public
planpri
noelec
coopele
sanitario1
sanitario2
sanitario3
sanitario5
sanitario6
energcocinar1
energcocinar2
energcocinar3
energcocinar4
elimbasu1
elimbasu2
elimbasu3
elimbasu4
elimbasu6
epared1
epared2
epared3
etecho1
etecho2
etecho3
eviv1
eviv2
eviv3
dis
male
female
estadocivil1
estadocivil2
estadocivil3
estadocivil4
estadocivil5
estadocivil6
estadocivil7
parentesco1
parentesco2
parentesco3
parentesco4
parentesco5
parentesco6
parentesco7
parentesco8
parentesco9
parentesco10
parentesco11
parentesco12
hogar_nin
hogar_adul
hogar_mayor
hogar_total
dependency
edjefe
edjefa
meaneduc
instlevel1
instlevel2
instlevel3
instlevel4
instl

In [25]:
list_kolom=X.columns.values.tolist()
for i in list_kolom:
    X[i]=(X[i]-X[i].min())/(X[i].max()-X[i].min())


## Applied Machine Learning Model

Pada tahap ini kita akan melakukan menerapkan model machine learning pada datasets. Kita akan mencoba tiga machine learning
model yaitu Logistic Regresion, KNN, dan SVM 

In [39]:
# Pembagian data training dan data test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/5, train_size=4/5, random_state=42)

### 1. Logistic Regression 

In [109]:
%%time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split


# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter space
C = np.logspace(0, 4, 10)
solver=['newton-cg','lbfgs']

logistic = LogisticRegression(max_iter=2000)
# Create hyperparameter options
hyperparameters = dict(C=C, solver=solver)
clf = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0,scoring='accuracy')
best_model = clf.fit(X_train, y_train)
print('Best Solver:', best_model.best_estimator_.get_params()['solver'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

Best Solver: newton-cg
Best C: 21.544346900318832
Wall time: 17min 22s


In [68]:
%%time
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=21.5, solver='newton-cg', max_iter=200)
print("Training Classifier...")
clf.fit(X_train, y_train)
print("Predicting...")
prediction = clf.predict(X_test)
accuracy = accuracy_score(y_test, prediction)
print ('Accuracy:', accuracy)

Training Classifier...
Predicting...
Accuracy: 0.6727672035139092
Wall time: 7.5 s


### 2. KNN 

In [83]:
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

n_neighbors=[3,4,5,7,9]
# Create hyperparameter options
hyperparameters = dict(n_neighbors=n_neighbors)
clf = GridSearchCV(KNeighborsClassifier(),scoring='accuracy', hyperparameters, cv=5, verbose=0)
best_model = clf.fit(X_train, y_train)
print('Best Neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])
#print('Best C:', best_model.best_estimator_.get_params()['C'])

Best Neighbors: 5
Wall time: 8min 52s


In [118]:
%%time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=5)
print("Training Classifier...")
clf.fit(X_train, y_train)
print("Predicting...")
prediction = clf.predict(X_test)
accuracy = accuracy_score(y_test, prediction)
print ('Accuracy:', accuracy)

# Menghitung nilai akurasi menggunakan metode Cross Validation 

print(cross_val_score(clf, X, y, cv=5)) # something wrong with this 

Training Classifier...
Predicting...
Accuracy: 0.7787656903765691
[0.62624151 0.61297071 0.54631083 0.46677132 0.45706806]
Wall time: 32.3 s


### 3. SVM 

In [100]:
%%time
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier


clf=OneVsRestClassifier(LinearSVC())
print("Training Classifier...")
clf.fit(X_train, y_train)
print("Predicting...")
prediction = clf.predict(X_test)
accuracy = accuracy_score(y_test, prediction)
print ('Accuracy:', accuracy)


Training Classifier...
Predicting...
Accuracy: 0.6698389458272328
Wall time: 12 s


### 4. Neural Network 

Grid Search untuk menentukan parameter terbaik untuk Neural Network 

In [40]:
%%time
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

mlp = MLPClassifier(random_state=1, learning_rate='adaptive')
hidden_layer_sizes=[20,50,66,88,100]
solver=['lbfgs','sgd','adam']
alpha=[1e-7,1e-5,1e-3,1e-2]
max_iter=[2000,4000]

# Create regularization hyperparameter space


# Create hyperparameter options
hyperparameters = dict(solver=solver, alpha=alpha, max_iter=max_iter, hidden_layer_sizes=hidden_layer_sizes)
clf = GridSearchCV(mlp, hyperparameters, cv=5, verbose=0,scoring='accuracy')
best_model = clf.fit(X_train, y_train)
print('Best hidden layer:', best_model.best_estimator_.get_params()['hidden_layer_sizes'])
print('Best solver:', best_model.best_estimator_.get_params()['solver'])
print('Best alpha:', best_model.best_estimator_.get_params()['alpha'])
print('Best iter:', best_model.best_estimator_.get_params()['max_iter'])

KeyboardInterrupt: 

In [42]:
%%time
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

clf = MLPClassifier(solver='adam', alpha=1e-5,
                    hidden_layer_sizes=(80,), random_state=1, learning_rate='adaptive',max_iter=2000)
print("Training Classifier...")
clf.fit(X_train, y_train)
print("Predicting...")
prediction = clf.predict(X_test)
accuracy = accuracy_score(y_test, prediction)
print ('Accuracy:', accuracy)


Training Classifier...
Predicting...
Accuracy: 0.8305439330543933
Wall time: 21.1 s


In [115]:
%%time
from sklearn.model_selection import validation_curve
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
clf = MLPClassifier(solver='adam', alpha=1e-7,
                    hidden_layer_sizes=(100), random_state=1, learning_rate='adaptive',max_iter=2000)

print(cross_val_score(clf, X, y, cv=5)) # needed to revise because something wrong with this
'''
print("Training Classifier...")
clf.fit(X_train, y_train)
print("Predicting...")
prediction = clf.predict(X_test)
accuracy = accuracy_score(y_test, prediction)
print ('Accuracy:', accuracy)
'''

[0.62415055 0.60303347 0.50758765 0.18524333 0.30732984]
Wall time: 4min 20s


In [94]:
df = pd.DataFrame(prediction, dtype='int64') # untuk mengubah data prediksi dari file menjadi type dataframe


In [121]:
X.shape

(9557, 127)

In [34]:
clf = MLPClassifier(solver='adam', alpha=1e-7,
                    hidden_layer_sizes=(100,), random_state=1, learning_rate='adaptive',max_iter=2000)
print("Predicting...")
clf.fit(X_train, y_train)
prediction = clf.predict(X)
accuracy = accuracy_score(y, prediction)
print ('Accuracy:', accuracy)

Predicting...
Accuracy: 0.9601339332426494


In [133]:
a=1e-5
b=1
a+b

1.00001

## Kesimpulan 

Tanpa menggunakan proses feature selection kita dapat mencapai akurasi 83% dengan menggunakan Neural Network. Untuk meningkatkan performa sistem kita akan menggunakan Feature Selection.